## 수치 미분을 이용한 심층 신경망 학습

## Import modules

In [12]:
import time 
import numpy as np
# tensorflow로 하면 이미 다 구현이 되어있기 때문에
# 굳이 구현할 필요가 없기때문에 numpy로 직접 수치 미분을 이용해서 구현해보자

## 유틸리티 함수

In [13]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def mean_squared_error(h, y):
    return 1 / 2 * np.mean(np.square(h - y))

## Dense Layer 구현

In [40]:
class Dense: # fully conected layer
    def __init__(self, W, b, a):
        self.W = W # weight metrix
        self.b = b # bias
        self.a = a # activation function
        
        # gradient 저장해주는 곳
        self.dW = np.zeros_like(self.W) # np.zeros_like() : 입력받은 것과 같은 shape, 자료형 가지고 0으로 초기화해줌
        self.db = np.zeros_like(self.b) 

    def __call__(self, x):
        # Dense Layer의 출력은 activation funtion을 최종으로 해주고
        # matrix 곱을 해줄 것, 정방향 곱
        return self.a(_m(_t(self.W), x) + self.b) # matmul((ixo)T ix1) + ox1

## 심층신경망 구현

In [44]:
class DNN:
    # hidden_depth : hidden layer 개수
    # num_neuron : 1개의 hidden layer 당 뉴런의 개수
    # num_input : input layer의 뉴런 개수
    # num_output : output layer의 뉴런 개수
    # activation : 무슨 activation 쓸건지
    
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            # i, o : input, output 변수 입력받아서 weight metrix와 bias 벡터를 어떻게 초기화 할건지
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))
                    # 평균 0, 표준편차 0.01인 sample들을 1000개 추출
            
        self.sequence = list()
        
        # First hidden layer
        # 입력을 hidden layer 개수로 바꿔주는 w matrix와 b 만들어줌
        W, b = init_var(num_input, num_neuron) # 초기화 된 W, b
        self.sequence.append(Dense(W, b, activation)) # 새로운 Danse layer append
        
        # Hidden layers
        # hidden_depth 개수만큼 Hidden layers 만들거야
        for _ in range(hidden_depth - 1): # 이미 하나 만들었기 때문에
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Dense(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Dense(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            # sequence의 layer를 불러와서 순차적으로 입력하게끔 구현
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        # numerical gradient 계산
        # 구현의 전략이 중요
        
        # 어떻게 값 하나를 바꿔서 Loss값을 한 번 구해줄 것이냐에 대한 func 정의
        # get_new_sequence func()은 값을 수정해서 새로운 sequence를 만들어줌
        # eval_sequence func()은 새로 만든 sequence를 가지고 평가
        
        def get_new_sequence(layer_index, new_layer):
            # 특정 index의 layer를 새로운 layer로 바꿔주는 func
            new_sequence = list()
            for i, layer in enumerate(self.sequence):
                if i == layer_index: # 바꾸고자 하는 index에 도달하게되면
                    new_sequence.append(new_layer) # 새로운 layer 입력
                else: 
                    new_sequence.append(layer) # 그 외에는 기존의 layer 넣어줌
            return new_sequence
        
        def eval_sequence(x, sequence):
            # 새로운 sequence 입력 받아서 출력값을 구할 수 있게끔
            for layer in sequence:
                x = layer(x)
            return x
        
        loss = loss_func(self(x), y) # 정답과 비교해 loss를 한번 구해
                        # self(x)는 위에 __call__을 하겠지, x 입력받은 걸 출력내줌
        
        # 각각의 모든 스칼라(variable:w,b)들로 각 스칼라들을 조금씩 값을 바꿔가면서 입력해주고 
        # loss와 비교해서 각각의 numerical gradient를 구해야하지
        # 그 말은 모든 스칼라들(w,b)을 한번씩 거쳐야 돼
        # 모든 layer를 iteration하고 그 안에 있는 모든 varibale(w, b)도 iteration해야 돼
        
        for layer_id, layer in enumerate(self.sequence): # enumerate : 현재 iteration하고 있는 것의 인덱스 return
                                # self.sequence의 layer와 몇 번째 layer인지까지 출력
            for w_i, w in enumerate(layer.W): # weight iteration
                # sequence에서 각 w를 반복하는데 w는 matrix이기때문에
                # 한번 돌면 행에 대해서 먼저 돔. 그래서 열에 대해서도 반복해줘야해
                for w_j, ww in enumerate(w):
                    # 이제 ww는 스칼라 하나가 됨
                    # 스칼라 하나만 수정해서 새로운 layer 만들거야
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon # i행의 j열 w에 스칼라 ww에 epsilon을 더해서 약간 수정된 값 넣어줌
                    
                    new_layer = Dense(W, layer.b, layer.a) # w는 새로 만든 것, b와 a는 이전 꺼 사용
                    new_seq = get_new_sequence(layer_id, new_layer) # 현재 layer id와 새로운 layer 입력해서 바꿔줌
                    h = eval_sequence(x, new_seq) # 새로운 sequence 평가
                    
                    num_grad = (loss_func(h, y) - loss) / epsilon
                    # (f(x+eps) - f(x)) / eps -> 이게 numerical gradient의 정의였지
                    layer.dW[w_i][w_j] = num_grad
                    
            for b_i, bb in enumerate(layer.b): # bias iteration
                # bias는 벡터이기 때문에 한 번 반복하면 스칼라 됨
                # ww와 bb는 스칼라
                
                b = np.copy(layer.b)
                b[b_i] = bb + epsilon 
                    
                new_layer = Dense(layer.W, b, layer.a)
                new_seq = get_new_sequence(layer_id, new_layer) 
                h = eval_sequence(x, new_seq)
                    
                num_grad = (loss_func(h, y) - loss) / epsilon
                layer.db[b_i] = num_grad
            
        return loss
    
# [정리]    
# numerical gradient를 구하기위해선 기준이 되는 loss를 구해야 돼 
# 학습을 하고자 하는 각각의 스칼라들을 epsilon만큼 옮겨서 loss function을 새로 구하고
# 그걸 기준이 되는 loss function과 차이를 구해주고 epsilon으로 나눠주면
# numerical gradient의 정의대로 구할 수 있음
# 그 해당 스칼라 위치에 numerical gradient 저장


## 경사하강 학습법

In [45]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = network.calc_gradient(x, y, loss_obj) # loss_object넣어서 gradient 계산
    for layer in network.sequence:
        # network의 각 layer마다 weight와 bias의 미분을 calc_gradient함수를 통해 계산해주면 
        # learning rate인 alpha로 학습
        
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss
# loss를 return 해주는 이유는 현재 학습이 얼만큼 되었는지 확인

## 동작 테스트

In [46]:
# x,y: 학습데이터 랜덤하게 만들어준 것, 성능이 좋게 나올 순 없겠지만 test로 사용
# np.random.normal(a, b, c) : 평균이 a, 표준편차가 b인 sample들 c개 추출
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

dnn = DNN(hidden_depth=5, num_neuron=32, num_input=10, num_output=2, activation=sigmoid)

t = time.time()
for epoch in range(100): # time check하면서 100 epochs까지
    loss = gradient_descent(dnn, x, y, mean_squared_error, 0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 1.2728720601682877
Epoch 1: Test loss 1.2657387273507668
Epoch 2: Test loss 1.2586515192559877
Epoch 3: Test loss 1.2516132702771037
Epoch 4: Test loss 1.2446266953222502
Epoch 5: Test loss 1.2376943847834099
Epoch 6: Test loss 1.2308188002531941
Epoch 7: Test loss 1.224002270990695
Epoch 8: Test loss 1.2172469911325468
Epoch 9: Test loss 1.2105550176362967
Epoch 10: Test loss 1.2039282689380197
Epoch 11: Test loss 1.1973685242995429
Epoch 12: Test loss 1.190877423816652
Epoch 13: Test loss 1.18445646905425
Epoch 14: Test loss 1.1781070242693956
Epoch 15: Test loss 1.1718303181830831
Epoch 16: Test loss 1.165627446257659
Epoch 17: Test loss 1.1594993734337378
Epoch 18: Test loss 1.1534469372829528
Epoch 19: Test loss 1.1474708515285355
Epoch 20: Test loss 1.1415717098896503
Epoch 21: Test loss 1.1357499902025763
Epoch 22: Test loss 1.1300060587763991
Epoch 23: Test loss 1.124340174939183
Epoch 24: Test loss 1.1187524957348705
Epoch 25: Test loss 1.1132430807323683
Ep

- 점점 loss가 줄어드는 모습을 보이지, 잘 학습되고 있다는 거야
- hidden_depth의 개수를 늘리면 어떻게 될까?
- numerical gradient의 문제가 DNN같은 경우, trainable parameter의 개수가 증가하면
- 그것의 제곱에 비례하게 학습 시간이 걸리는 게 문제
- 따라서 만약 hidden_depth 개수를 2배 증가하면, 학습시간은 4배 증가함